In [25]:
# Penjelasan Dataset
# Dataset ini berisi tentang data nasabah bank yang akan memutuskan untuk berhenti atau tidak dari bank tersebut dalam 6 bulan terakhir
# Kolom-kolom dalam dataset :
# RowNumber : Nomor baris
# CustomerId : Nomor ID nasabah 
# Surname : Nama belakang nasabah
# CreditScore : Skor kredit nasabah (int)
# Geography : Negara asal nasabah (kategorikal)
# Gender : Jenis kelamin nasabah (kategorikal)
# Age : Umur nasabah (int)
# Tenure : Jumlah tahun nasabah menjadi nasabah bank (int)
# Balance : Jumlah saldo nasabah (float)
# NumOfProducts : Jumlah produk yang digunakan nasabah (int)
# HasCrCard : Apakah nasabah memiliki kartu kredit (biner)
# IsActiveMember : Apakah nasabah aktif (biner)
# EstimatedSalary : Gaji nasabah (float)
# Exited : Apakah nasabah keluar dari bank (biner)

# Tujuan
# Membuat model ANN untuk memprediksi apakah nasabah akan keluar dari bank atau tidak


In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [27]:
# Data Preprocessing
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough') # ada 3 kolom variabel encoding
X = np.array(ct.fit_transform(X))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state = 0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [28]:
# Membuat FNN dengan Tensorflow
ann = tf.keras.models.Sequential() # inisiasi model
# keras adalah API dari tensorflow
# sequential adalah tipe model yang digunakan untuk membuat model ANN

ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) # menambah hidden layer pertama (input layer dimasukkan secara otomatis)
# units = jumlah neuron pada hidden layer
# activation = fungsi aktivasi yang digunakan pada hidden layer (relu = rectifier linear unit)

ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) # menambah hidden layer kedua

ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) # menambah output layer
# sigmoid adalah fungsi aktivasi yang baik untuk klasifikasi biner
# fungsi aktivasi lainnya :
# softmax = fungsi aktivasi yang baik untuk klasifikasi multiclass
# tanh = fungsi aktivasi yang baik untuk klasifikasi biner

In [29]:
# Melatih model FNN

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # compile model
# optimizer = algoritma yang digunakan untuk menentukan bobot optimal (adam = adaptive moment estimation)
# loss = fungsi loss yang digunakan untuk menghitung error (binary_crossentropy = binary classification)
# fungsi loss lainnya : 
# categorical_crossentropy = digunakan dalam multiclass classification
# metrics = metrik yang digunakan untuk mengukur performa model (accuracy = akurasi)

ann.fit(X_train, y_train, batch_size = 32, epochs = 100) # melatih model
# batch_size = jumlah data yang akan diproses pada setiap epoch
# epoch : satu kali proses forward dan backward propagation


Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5939 - accuracy: 0.7216
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4702 - accuracy: 0.7969: 0s - loss: 0.5058 - ac
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4423 - accuracy: 0.8055
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4283 - accuracy: 0.8108
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4184 - accuracy: 0.8198
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4108 - accuracy: 0.8239
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4047 - accuracy: 0.8290
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3996 - accuracy: 0.8298
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3954 - accuracy: 0.8299
Epoch 10/100
250/250 [==============================] - 0s 1ms/step

In [30]:
# memprediksi data tunggal
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5) # memprediksi data tunggal

[[False]]


In [31]:
# Evaluasi akurasi test set
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5) # mengubah nilai probabilitas menjadi biner
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)) # menampilkan hasil prediksi dan nilai aktual

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))


[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
[[1524   71]
 [ 206  199]]
0.8615
